In [ ]:
import os
from pathlib import Path
import numpy as np
import rasterio
from rasterio.enums import Resampling
from tensorflow.keras import layers, models, callbacks
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib
from tqdm import tqdm

# === CONFIG ===
SNOW_DIR = Path(r"C:\mnarimani\1-UCDavis\16-ESEARCH\2-Clean_Data\Snow")
NOT_SNOW_DIR = Path(r"C:\mnarimani\1-UCDavis\16-ESEARCH\2-Clean_Data\Not_Snow")
OUT_MASK_DIR = SNOW_DIR.parent / "Anomaly_Masks"
OUT_MASK_DIR.mkdir(exist_ok=True)

BANDS = 12
PERCENTILE = 95  # For thresholding
EPOCHS = 40
BATCH = 8192
LATENT_DIM = 8
THRESHOLD_METHOD = 'percentile'  # or 'mean+2std'

# === HELPERS ===
def read_all_pixels_from_folder(folder, max_images=100):
    """Reads all pixels from all .tif files in a folder, returns a (N, BANDS) array."""
    pixel_list = []
    for i, tif in enumerate(sorted(folder.glob("*.tif"))):
        if i >= max_images:
            break
        with rasterio.open(tif) as src:
            arr = src.read(list(range(1, BANDS+1))).astype('float32') / 10000.0  # (12, H, W)
            arr = arr.reshape(BANDS, -1).T  # (H*W, 12)
            nodata = src.nodata
            if nodata is not None:
                mask = np.any(arr == nodata, axis=1)
                arr = arr[~mask]
            arr = arr[~np.isnan(arr).any(axis=1)]
            pixel_list.append(arr)
    if pixel_list:
        X = np.vstack(pixel_list)
    else:
        X = np.empty((0, BANDS))
    return X

# === 1. LOAD DATA ===
print("Loading Snow spectra...")
X_snow = read_all_pixels_from_folder(SNOW_DIR)
print(f"  Got {X_snow.shape[0]:,} pixels")

print("Loading Not Snow spectra...")
X_not_snow = read_all_pixels_from_folder(NOT_SNOW_DIR)
print(f"  Got {X_not_snow.shape[0]:,} pixels")

# === 2. PREPROCESSING ===
imputer = SimpleImputer(strategy="median")
X_snow_imp = imputer.fit_transform(X_snow)

scaler = StandardScaler()
X_snow_scaled = scaler.fit_transform(X_snow_imp)

# === 3. BUILD AUTOENCODER ===
input_layer = layers.Input(shape=(BANDS,))
x = layers.Dense(32, activation="relu")(input_layer)
x = layers.Dense(16, activation="relu")(x)
encoded = layers.Dense(LATENT_DIM, activation="relu")(x)
x = layers.Dense(16, activation="relu")(encoded)
x = layers.Dense(32, activation="relu")(x)
decoded = layers.Dense(BANDS, activation="linear")(x)
autoencoder = models.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

# === 4. TRAIN AUTOENCODER ON SNOW ONLY ===
autoencoder.fit(X_snow_scaled, X_snow_scaled,
                epochs=EPOCHS,
                batch_size=BATCH,
                shuffle=True,
                validation_split=0.05,
                verbose=2,
                callbacks=[callbacks.EarlyStopping(patience=4, restore_best_weights=True)])

# === 5. GET RECONSTRUCTION ERROR DISTRIBUTION (ON SNOW) ===
recon_snow = autoencoder.predict(X_snow_scaled, batch_size=BATCH, verbose=1)
err_snow = np.mean((recon_snow - X_snow_scaled) ** 2, axis=1)
if THRESHOLD_METHOD == 'percentile':
    thr = np.percentile(err_snow, PERCENTILE)
elif THRESHOLD_METHOD == 'mean+2std':
    thr = err_snow.mean() + 2*err_snow.std()
print(f"Anomaly threshold (95th percentile): {thr:.6g}")

# === 6. FUNCTION TO GENERATE MASK FOR AN IMAGE ===
def make_anomaly_mask(tif_path):
    with rasterio.open(tif_path) as src:
        arr = src.read(list(range(1, BANDS+1))).astype('float32') / 10000.0  # (12, H, W)
        h, w = arr.shape[1], arr.shape[2]
        arr = arr.reshape(BANDS, -1).T  # (H*W, 12)
        nodata = src.nodata
        mask_valid = np.ones(arr.shape[0], dtype=bool)
        if nodata is not None:
            mask_valid = ~np.any(arr == nodata, axis=1)
        arr_imp = imputer.transform(arr)
        arr_scaled = scaler.transform(arr_imp)
        recon = autoencoder.predict(arr_scaled, batch_size=8192, verbose=0)
        err = np.mean((recon - arr_scaled)**2, axis=1)
        mask = np.zeros(arr.shape[0], dtype=np.uint8)
        mask[mask_valid] = (err[mask_valid] < thr).astype(np.uint8)
        mask_img = mask.reshape(h, w)
        return mask_img, src.profile

# === 7. GENERATE MASKS FOR ALL IMAGES AND SAVE ===
def process_folder(in_folder, out_folder):
    for tif in tqdm(sorted(in_folder.glob("*.tif"))):
        mask_img, prof = make_anomaly_mask(tif)
        prof.update(count=1, dtype='uint8', nodata=0, compress='lzw')
        out_path = out_folder / (tif.stem + "_mask.tif")
        with rasterio.open(out_path, "w", **prof) as dst:
            dst.write(mask_img, 1)
        print(f"Saved: {out_path}")

print("Generating masks for Snow images...")
process_folder(SNOW_DIR, OUT_MASK_DIR)
print("Generating masks for Not Snow images...")
process_folder(NOT_SNOW_DIR, OUT_MASK_DIR)

# === 8. SAVE MODEL AND PREPROCESSORS ===
autoencoder.save(str(OUT_MASK_DIR / "snow_autoencoder.h5"))
joblib.dump({"imputer": imputer, "scaler": scaler, "thr": thr}, OUT_MASK_DIR / "snow_ae_preprocessing.pkl")
print("✓ All done! Masks, model, and preprocessors saved.")

In [ ]:
import numpy as np
import rasterio
import joblib
from pathlib import Path
from tensorflow.keras.models import load_model
from tqdm import tqdm

# === PATHS ===
SNOW_DIR = Path(r"C:\mnarimani\1-UCDavis\16-ESEARCH\2-Clean_Data\Snow")
NOT_SNOW_DIR = Path(r"C:\mnarimani\1-UCDavis\16-ESEARCH\2-Clean_Data\Not_Snow")
OUT_SCORE_DIR = SNOW_DIR.parent / "Anomaly_Score_Maps_V2"
OUT_SCORE_DIR.mkdir(exist_ok=True)

MODEL_PATH = SNOW_DIR.parent / "Anomaly_Masks" / "snow_autoencoder.h5"
PREP_PATH = SNOW_DIR.parent / "Anomaly_Masks" / "snow_ae_preprocessing.pkl"

BANDS = 12

# === LOAD MODEL AND PREPROCESSORS ===
autoencoder = load_model(MODEL_PATH)
prep = joblib.load(PREP_PATH)
imputer = prep["imputer"]
scaler = prep["scaler"]

# For score normalization:
if "err_snow_sorted" in prep:
    err_snow_sorted = prep["err_snow_sorted"]
    N_h = len(err_snow_sorted)
else:
    # Recompute using Snow pixels 
    print("No err_snow_sorted found. Will recompute on the fly if needed.")

# === COMPUTE CDF FOR ERROR (FROM TRAIN SNOW PIXELS) ===

def anomaly_score_from_error(err, err_snow_sorted):
    idx = np.searchsorted(err_snow_sorted, err, side="right")
    score = np.clip(idx / len(err_snow_sorted), 0, 1)
    return score

# === SCORE MAP FUNCTION ===
def make_anomaly_score_map(tif_path, err_snow_sorted):
    with rasterio.open(tif_path) as src:
        arr = src.read(list(range(1, BANDS+1))).astype('float32') / 10000.0  # (12, H, W)
        h, w = arr.shape[1], arr.shape[2]
        arr = arr.reshape(BANDS, -1).T  # (H*W, 12)
        nodata = src.nodata
        mask_valid = np.ones(arr.shape[0], dtype=bool)
        if nodata is not None:
            mask_valid = ~np.any(arr == nodata, axis=1)
        arr_imp = imputer.transform(arr)
        arr_scaled = scaler.transform(arr_imp)
        recon = autoencoder.predict(arr_scaled, batch_size=8192, verbose=0)
        err = np.mean((recon - arr_scaled)**2, axis=1)
        score = np.zeros(arr.shape[0], dtype=np.float32)
        score[mask_valid] = anomaly_score_from_error(err[mask_valid], err_snow_sorted)
        score_img = score.reshape(h, w)
        return score_img, src.profile

# === LOAD SNOW PIXELS TO GET err_snow_sorted ===
def read_all_pixels_from_folder(folder, max_images=100):
    pixel_list = []
    for i, tif in enumerate(sorted(folder.glob("*.tif"))):
        if i >= max_images:
            break
        with rasterio.open(tif) as src:
            arr = src.read(list(range(1, BANDS+1))).astype('float32') / 10000.0  # (12, H, W)
            arr = arr.reshape(BANDS, -1).T  # (H*W, 12)
            nodata = src.nodata
            if nodata is not None:
                mask = np.any(arr == nodata, axis=1)
                arr = arr[~mask]
            arr = arr[~np.isnan(arr).any(axis=1)]
            pixel_list.append(arr)
    if pixel_list:
        X = np.vstack(pixel_list)
    else:
        X = np.empty((0, BANDS))
    return X

print("Building snow error CDF for scoring...")
X_snow = read_all_pixels_from_folder(SNOW_DIR)
X_snow_imp = imputer.transform(X_snow)
X_snow_scaled = scaler.transform(X_snow_imp)
recon_snow = autoencoder.predict(X_snow_scaled, batch_size=8192, verbose=1)
err_snow = np.mean((recon_snow - X_snow_scaled) ** 2, axis=1)
err_snow_sorted = np.sort(err_snow)

# === PROCESS FOLDERS ===
def process_folder_score(in_folder, out_folder):
    for tif in tqdm(sorted(in_folder.glob("*.tif"))):
        score_img, prof = make_anomaly_score_map(tif, err_snow_sorted)
        prof.update(count=1, dtype='float32', nodata=np.nan, compress='lzw')
        out_path = out_folder / (tif.stem + "_score.tif")
        with rasterio.open(out_path, "w", **prof) as dst:
            dst.write(score_img, 1)
        print(f"Saved: {out_path}")

print("Generating continuous anomaly score maps for Snow images...")
process_folder_score(SNOW_DIR, OUT_SCORE_DIR)
print("Generating continuous anomaly score maps for Not Snow images...")
process_folder_score(NOT_SNOW_DIR, OUT_SCORE_DIR)

print("✓ All done! Continuous anomaly score maps saved.")

In [ ]:
import os
from pathlib import Path

# === PATHS ===
SNOW_DIR = Path(r"C:\mnarimani\1-UCDavis\16-ESEARCH\2-Clean_Data\Snow")
NOT_SNOW_DIR = Path(r"C:\mnarimani\1-UCDavis\16-ESEARCH\2-Clean_Data\Not_Snow")
SCORE_MAPS_DIR = Path(r"C:\mnarimani\1-UCDavis\16-ESEARCH\2-Clean_Data\Anomaly_Score_Maps")

# --- RENAME SNOW FILES ---
snow_files = sorted(SNOW_DIR.glob("Square_*.tif"))
for idx, file in enumerate(snow_files, 1):
    new_name = f"Snow_{idx}.tif"
    new_path = file.with_name(new_name)
    print(f"Renaming {file.name} -> {new_name}")
    file.rename(new_path)

# --- RENAME NOT SNOW FILES ---
not_snow_files = sorted(NOT_SNOW_DIR.glob("Square_*.tif"))
for idx, file in enumerate(not_snow_files, 1):
    new_name = f"Not_Snow_{idx}.tif"
    new_path = file.with_name(new_name)
    print(f"Renaming {file.name} -> {new_name}")
    file.rename(new_path)

# --- RENAME ANOMALY SCORE MAPS (MATCHING TO SNOW AND NOT SNOW) ---

# For Snow
snow_rename_map = {}
for idx, old_file in enumerate(sorted(SCORE_MAPS_DIR.glob("Square*_score.tif")), 1):
    # Check if original file is in SNOW_DIR
    orig_file = SNOW_DIR.parent / "Snow" / f"Snow_{idx}.tif"
    new_score_name = f"Snow_{idx}_Score.tif"
    snow_rename_map[old_file.name] = new_score_name

# For Not Snow
not_snow_rename_map = {}
for idx, old_file in enumerate(sorted(SCORE_MAPS_DIR.glob("Square*_score.tif")), 1):
    # Check if original file is in NOT_SNOW_DIR
    orig_file = NOT_SNOW_DIR.parent / "Not_Snow" / f"Not_Snow_{idx}.tif"
    new_score_name = f"Not_Snow_{idx}_Score.tif"
    not_snow_rename_map[old_file.name] = new_score_name

# Now rename all score maps, using the presence of a file in the original directory as the criterion.
score_files = sorted(SCORE_MAPS_DIR.glob("Square*_score.tif"))
for idx, score_file in enumerate(score_files, 1):
    # Try Snow first:
    if idx <= len(snow_files):
        new_score_name = f"Snow_{idx}_Score.tif"
    else:
        new_score_name = f"Not_Snow_{idx - len(snow_files)}_Score.tif"
    new_score_path = score_file.with_name(new_score_name)
    print(f"Renaming {score_file.name} -> {new_score_name}")
    score_file.rename(new_score_path)